In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv('../../datasets/titanic/train.csv')
df_test   = pd.read_csv('../../datasets/titanic/test.csv')

In [3]:
df_train.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


# Feature Engineering

# Missing Data -> fillna

In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
# age
df_train['Age'].mean(), df_train['Age'].median()

(29.69911764705882, 28.0)

In [6]:
df_train['Age'].fillna(df_train['Age'].median(), inplace=True)
df_test['Age'].fillna(df_train['Age'].median(), inplace=True)

In [7]:
# 2. Embarked -> mode
df_train['Embarked'].mode().iloc[0]

'S'

In [8]:
df_train['Embarked'].fillna(df_train['Embarked'].mode().iloc[0], inplace=True)
df_test['Embarked'].fillna(df_train['Embarked'].mode().iloc[0], inplace=True)

In [9]:
# 3. Fare -> median
df_train['Fare'].mean(), df_train['Fare'].median()

(32.2042079685746, 14.4542)

In [10]:
df_train['Fare'].fillna(df_train['Fare'].median(), inplace=True)
df_test['Fare'].fillna(df_train['Fare'].median(), inplace=True)


In [11]:
# 4. Cabin
df_train['Cabin'] = (~df_train['Cabin'].isna()).astype(int)
df_test['Cabin'] = (~df_test['Cabin'].isna()).astype(int)

In [12]:
df_train.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

# Binning

In [13]:
def get_age_cat(age):
    if age<18:
        return 'child'
    if age<50:
        return 'young'
    return 'old'

df_train['Age_Cat'] = df_train['Age'].apply(get_age_cat)
df_test['Age_Cat'] = df_test['Age'].apply(get_age_cat)

In [14]:
df_train.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_Cat
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,0,S,young
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,1,C,young


# Encoding

In [15]:
# Age_Cat, Sex, Embarked
df_train['Sex'] = df_train['Sex'].map({'male':0, 'female':1})
df_test['Sex'] = df_test['Sex'].map({'male':0, 'female':1})

df_train['Embarked'] = df_train['Embarked'].map({'S':0, 'C':1, 'Q':2})
df_test['Embarked'] = df_test['Embarked'].map({'S':0, 'C':1, 'Q':2})

df_train['Age_Cat'] = df_train['Age_Cat'].map({'child':0, 'young':1, 'old':2})
df_test['Age_Cat'] = df_test['Age_Cat'].map({'child':0, 'young':1, 'old':2})

In [16]:
# name
combine = [df_train,df_test]
for dataset in combine:
    dataset['Title'] = dataset['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

In [17]:
title_mapping = {"Mr": 0, "Miss": 1, "Mrs": 2, 
                 "Master": 3, "Dr": 3, "Rev": 3, "Col": 3, "Major": 3, "Mlle": 3,"Countess": 3,
                 "Ms": 3, "Lady": 3, "Jonkheer": 3, "Don": 3, "Dona" : 3, "Mme": 3,"Capt": 3,"Sir": 3 }

for dataset in combine:
    dataset['Title'] = dataset["Title"].map(title_mapping)

In [18]:
df_train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_Cat,Title
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,0,0,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,1,1,1,2
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,0,0,1,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,1,0,1,2
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,0,0,1,0


In [19]:
df_train['Title'].isna()

0      False
1      False
2      False
3      False
4      False
       ...  
886    False
887    False
888    False
889    False
890    False
Name: Title, Length: 891, dtype: bool

In [20]:
df_train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_Cat,Title
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,0,0,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,1,1,1,2
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,0,0,1,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,1,0,1,2
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,0,0,1,0


In [21]:
selected_columns = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked', 'Age_Cat','Fare','Title']

# Assign x ,y

In [22]:
x = df_train[selected_columns]
y = df_train['Survived']

# Model Building

In [23]:
from sklearn.model_selection import train_test_split
xtrain, xval , ytrain, yval = train_test_split(x,y,test_size=0.22,random_state=18)

In [24]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=10 , random_state=100)
model.fit(xtrain,ytrain)

RandomForestClassifier(n_estimators=10, random_state=100)

In [25]:
model.score(xtrain,ytrain)

0.9380403458213257

# hyperparameter Tuning : 1.random

In [26]:
from sklearn.model_selection import RandomizedSearchCV

In [27]:
params = {'max_depth':[1,2,3,4,5],
          'min_samples_leaf':[1,2,3,4],
          'min_samples_split':[3,4,5,6]}

In [28]:
random_cv = RandomizedSearchCV(RandomForestClassifier(random_state=100),
                              param_distributions=params,cv= 4 ,verbose=2)
random_cv.fit(x,y)

Fitting 4 folds for each of 10 candidates, totalling 40 fits
[CV] END max_depth=2, min_samples_leaf=3, min_samples_split=5; total time=   0.3s
[CV] END max_depth=2, min_samples_leaf=3, min_samples_split=5; total time=   0.3s
[CV] END max_depth=2, min_samples_leaf=3, min_samples_split=5; total time=   0.3s
[CV] END max_depth=2, min_samples_leaf=3, min_samples_split=5; total time=   0.2s
[CV] END max_depth=3, min_samples_leaf=2, min_samples_split=6; total time=   0.3s
[CV] END max_depth=3, min_samples_leaf=2, min_samples_split=6; total time=   0.3s
[CV] END max_depth=3, min_samples_leaf=2, min_samples_split=6; total time=   0.3s
[CV] END max_depth=3, min_samples_leaf=2, min_samples_split=6; total time=   0.3s
[CV] END max_depth=1, min_samples_leaf=1, min_samples_split=4; total time=   0.3s
[CV] END max_depth=1, min_samples_leaf=1, min_samples_split=4; total time=   0.2s
[CV] END max_depth=1, min_samples_leaf=1, min_samples_split=4; total time=   0.2s
[CV] END max_depth=1, min_samples_lea

RandomizedSearchCV(cv=4, estimator=RandomForestClassifier(random_state=100),
                   param_distributions={'max_depth': [1, 2, 3, 4, 5],
                                        'min_samples_leaf': [1, 2, 3, 4],
                                        'min_samples_split': [3, 4, 5, 6]},
                   verbose=2)

In [29]:
random_cv.best_score_

0.8192946309538238

In [30]:
random_cv.best_index_

4

# Make Predictions

In [31]:
yp = random_cv.predict(df_test[selected_columns])

In [32]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    int64  
 4   Age          418 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         418 non-null    float64
 9   Cabin        418 non-null    int32  
 10  Embarked     418 non-null    int64  
 11  Age_Cat      418 non-null    int64  
 12  Title        418 non-null    int64  
dtypes: float64(2), int32(1), int64(8), object(2)
memory usage: 40.9+ KB


# Submission

In [33]:
df_test['Survived'] = yp

In [34]:
df_test[['PassengerId','Survived']].to_csv('onkar7.csv', index=False)